<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Q%26A_Chat_Bot/poly_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
!pip install sentence_transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 4.4 MB 35.0 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f715f4a5912412fdc7214e56e8f63233c8413d679a5dd3c63f8086a2df5ad674
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [2]:
import urllib.request

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertPreTrainedModel, BertModel
from transformers import BertModel, BertConfig, BertTokenizer, BertTokenizerFast

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

# Load Data

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# file_path = '/content/drive/MyDrive/알파코/장기 프로젝트/챗봇/test_file.csv'
# df = pd.read_csv(file_path)
# df.drop(columns='Unnamed: 0', inplace=True)

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
df = pd.read_csv('ChatBotData.csv')
df.drop(columns='label', inplace=True)

In [5]:
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


# Load Model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
config = AutoConfig.from_pretrained('jhgan/ko-sroberta-multitask')

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

In [7]:
config.hidden_size = int(768/4)
model = AutoModel.from_config(config)

# Encoder

In [8]:
class Poly_Embedding(BertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = kwargs['bert']

    def forward(self, responses_input_ids, responses_input_masks):
            
        batch_size, res_cnt, seq_length = responses_input_ids.unsqueeze(1).shape 

        # response encoder
        responses_input_ids = responses_input_ids.view(-1, seq_length)
        responses_input_masks = responses_input_masks.view(-1, seq_length)
        cand_emb = self.bert(responses_input_ids, responses_input_masks)[0][:,0,:]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1)

        return cand_emb

In [9]:
Poly_Embedding = Poly_Embedding(config=config, bert=model)
Poly_Embedding.resize_token_embeddings(len(tokenizer))

Embedding(32000, 192, padding_idx=1)

In [10]:
embedding = []

for response in tqdm(df['Q']):

  response_input = tokenizer(response, padding=True, truncation=True, return_tensors='pt')
  responses_input_ids, responses_input_masks = response_input['input_ids'], response_input['attention_mask']

  cand_emb = Poly_Embedding(responses_input_ids, responses_input_masks)
  cand_emb = cand_emb.detach().numpy()
  
  embedding.append(cand_emb)

  0%|          | 0/11823 [00:00<?, ?it/s]

In [11]:
df['embedding'] = embedding

In [12]:
df.head()

,Q,A,embedding
0,12시 땡!,하루가 또 가네요.,"[[[-0.5124754, -0.70941406, -0.46491793, -0.63..."
1,1지망 학교 떨어졌어,위로해 드립니다.,"[[[-0.49622366, -0.61735356, 0.072931185, -0.7..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[[[-0.8368761, -0.8404734, -0.7721939, -0.7253..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,"[[[0.051838025, -0.54713875, -0.5649379, -0.30..."
4,PPL 심하네,눈살이 찌푸려지죠.,"[[[0.2634862, -0.48078385, -0.47905913, -0.491..."


In [13]:
class PolyEncoder(BertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = kwargs['bert']
        self.poly_m = kwargs['poly_m']
        self.poly_code_embeddings = nn.Embedding(self.poly_m, config.hidden_size)
        torch.nn.init.normal_(self.poly_code_embeddings.weight, config.hidden_size ** -0.5)

    def dot_attention(self, q, k, v):

        attn_weights = torch.matmul(q, k.transpose(2, 1)) 
        attn_weights = F.softmax(attn_weights, -1)
        output = torch.matmul(attn_weights, v) 
        return output

    def forward(self, context_input_ids, context_input_masks, cand_emb, labels=None):
            
        batch_size, res_cnt, seq_length = responses_input_ids.unsqueeze(1).shape 

        # context encoder
        ctx_out = self.bert(context_input_ids, context_input_masks)[0]  
        poly_code_ids = torch.arange(self.poly_m, dtype=torch.long).to(context_input_ids.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(batch_size, self.poly_m)
        poly_codes = self.poly_code_embeddings(poly_code_ids) 
        embs = self.dot_attention(poly_codes, ctx_out, ctx_out) 

        # response encoder
        cand_emb = cand_emb

        ctx_emb = self.dot_attention(cand_emb, embs, embs) 
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

In [14]:
model = PolyEncoder(config=config, bert=model, poly_m=16)
model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 192, padding_idx=1)

In [15]:
def calculaxcte_score(context_input_ids, context_input_masks, emb):
  
  cand_emb = torch.Tensor(emb)

  dot_product = model(context_input_ids, context_input_masks, cand_emb)

  score = dot_product.detach().numpy()[0][0]

  return score

In [16]:
def return_answer(question):
  
  context_input = tokenizer(question, padding=True, truncation=True, return_tensors='pt')
  context_input_ids, context_input_masks = context_input['input_ids'], context_input['attention_mask']

  score_list = []

  for response in tqdm(df['Q']):

    score = calculaxcte_score(context_input_ids, context_input_masks, cand_emb)

    score_list.append(score)

  df['score'] = score_list

  return df.loc[df['score'].idxmax()]['A']

In [17]:
# def return_answer(question):
  
#   context_input = tokenizer(question, padding=True, truncation=True, return_tensors='pt')
#   context_input_ids, context_input_masks = context_input['input_ids'], context_input['attention_mask']

#   df['score'] = df.apply(lambda x : calculaxcte_score(context_input_ids, context_input_masks, torch.Tensor(x['embedding'])), axis=1)

#   return df.loc[df['score'].idxmax()]['A']

# Test

In [18]:
return_answer('너가 좋아')

  0%|          | 0/11823 [00:00<?, ?it/s]

'좋은 태도네요.'

In [19]:
return_answer('결혼하고싶어')

  0%|          | 0/11823 [00:00<?, ?it/s]

'습관처럼은 되지 않길 바랄게요.'

In [20]:
return_answer('나랑 커피먹을래?')

  0%|          | 0/11823 [00:00<?, ?it/s]

'이상형은 아니지만 호감가는 사람이 있어요.'

In [21]:
return_answer('반가워')

  0%|          | 0/11823 [00:00<?, ?it/s]

'놀러가세요!'